In [1]:
import torch

In [2]:
model_p = "./swint_upernet_imp_sep_model.pth"
a = torch.load(model_p)

In [3]:
a.keys()

dict_keys(['epoch', 'iteration', 'state_dict', 'optimizer', 'scheduler', 'loss_pretrain'])

In [4]:
swin_uper_keys = a["state_dict"].keys()

In [5]:
[i for i in swin_uper_keys]

['encoder.patch_embed.proj.weight',
 'encoder.patch_embed.proj.bias',
 'encoder.patch_embed.norm.weight',
 'encoder.patch_embed.norm.bias',
 'encoder.layers.0.blocks.0.norm1.weight',
 'encoder.layers.0.blocks.0.norm1.bias',
 'encoder.layers.0.blocks.0.attn.relative_position_bias_table',
 'encoder.layers.0.blocks.0.attn.relative_position_index',
 'encoder.layers.0.blocks.0.attn.qkv.weight',
 'encoder.layers.0.blocks.0.attn.qkv.bias',
 'encoder.layers.0.blocks.0.attn.proj.weight',
 'encoder.layers.0.blocks.0.attn.proj.bias',
 'encoder.layers.0.blocks.0.norm2.weight',
 'encoder.layers.0.blocks.0.norm2.bias',
 'encoder.layers.0.blocks.0.mlp.fc1.weight',
 'encoder.layers.0.blocks.0.mlp.fc1.bias',
 'encoder.layers.0.blocks.0.mlp.fc2.weight',
 'encoder.layers.0.blocks.0.mlp.fc2.bias',
 'encoder.layers.0.blocks.1.norm1.weight',
 'encoder.layers.0.blocks.1.norm1.bias',
 'encoder.layers.0.blocks.1.attn.relative_position_bias_table',
 'encoder.layers.0.blocks.1.attn.relative_position_index',
 'en

In [6]:
import swin

In [7]:
model_swin = swin.swin()

/mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
dummy = torch.randn(4,3,224,224)

In [9]:
out_ = model_swin(dummy)

In [10]:
out_[4].shape

torch.Size([4, 768, 7, 7])

In [11]:
model_swin.out_channels[1:]

(96, 192, 384, 768)

In [12]:
import upper_net_mmseg

In [13]:
model_upernet = upper_net_mmseg.UPerHead(
    in_channels = model_swin.out_channels[1:],
    channels = model_swin.out_channels[2],
    in_index= (0,1,2,3),
    dropout_ratio=0.1,
    norm_cfg= dict(type='SyncBN', requires_grad=True)
)

In [14]:
model_upernet

UPerHead(
  input_transform=multiple_select, ignore_index=255, align_corners=False
  (dropout): Dropout2d(p=0.1, inplace=False)
  (psp_modules): PPM(
    (0): Sequential(
      (0): AdaptiveAvgPool2d(output_size=1)
      (1): ConvModule(
        (conv): Conv2d(768, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): SyncBatchNorm(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): ReLU(inplace=True)
      )
    )
    (1): Sequential(
      (0): AdaptiveAvgPool2d(output_size=2)
      (1): ConvModule(
        (conv): Conv2d(768, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): SyncBatchNorm(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): ReLU(inplace=True)
      )
    )
    (2): Sequential(
      (0): AdaptiveAvgPool2d(output_size=3)
      (1): ConvModule(
        (conv): Conv2d(768, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): SyncBatchNorm(192, eps=1e-05, mo

In [15]:
#     print('################# Using UNet for Pretraining! ######################')

# elif args.decoder == 'upernet':

#     self.decoder = UPerHead(
#         in_channels = self.encoder.out_channels[1:],
#         channels = self.encoder.out_channels[2],
#         in_index = (0, 1, 2, 3),
#         dropout_ratio = 0.1,
#         norm_cfg = dict(type='SyncBN', requires_grad=True)
#     )


In [16]:
import torch.nn as nn 

class SamRS(torch.nn.Module):
    def __init__(self, 
                model1=None,
                model2=None, 
                decoder_use_batchnorm: bool = True,
                decoder_channels  = (512, 256, 128, 64), #(256, 128, 64, 32, 16),
                decoder_attention_type = None,
                classes1: int = 18,
                classes2: int = 20,
                classes3: int = 37,
                activation: str = None,
                aux_params: dict = None):
        
        super(SamRS,self).__init__()
        
        self.encoder = model1 
        self.decoder = model2
        self.semseghead_1 = nn.Sequential(
                nn.Dropout2d(0.1),
                nn.Conv2d(self.encoder.out_channels[2], classes1, kernel_size=1)
            )

        self.semseghead_2 = nn.Sequential(
                nn.Dropout2d(0.1),
                nn.Conv2d(self.encoder.out_channels[2], classes2, kernel_size=1)
            )

        self.semseghead_3 = nn.Sequential(
                nn.Dropout2d(0.1),
                nn.Conv2d(self.encoder.out_channels[2], classes3, kernel_size=1)
            )
    
    def forward(self,x):
        features = self.encoder(x)
        output = self.decoder(*features)
        output_1 = self.semseghead_1(output)
        output_2 = self.semseghead_2(output)
        output_3 = self.semseghead_3(output)
        
        return output_3

#---------------------------------------    
            

In [17]:
samrs_model = SamRS(model1=model_swin, model2=model_upernet)

In [18]:
[i for i in a['state_dict'].keys()]

['encoder.patch_embed.proj.weight',
 'encoder.patch_embed.proj.bias',
 'encoder.patch_embed.norm.weight',
 'encoder.patch_embed.norm.bias',
 'encoder.layers.0.blocks.0.norm1.weight',
 'encoder.layers.0.blocks.0.norm1.bias',
 'encoder.layers.0.blocks.0.attn.relative_position_bias_table',
 'encoder.layers.0.blocks.0.attn.relative_position_index',
 'encoder.layers.0.blocks.0.attn.qkv.weight',
 'encoder.layers.0.blocks.0.attn.qkv.bias',
 'encoder.layers.0.blocks.0.attn.proj.weight',
 'encoder.layers.0.blocks.0.attn.proj.bias',
 'encoder.layers.0.blocks.0.norm2.weight',
 'encoder.layers.0.blocks.0.norm2.bias',
 'encoder.layers.0.blocks.0.mlp.fc1.weight',
 'encoder.layers.0.blocks.0.mlp.fc1.bias',
 'encoder.layers.0.blocks.0.mlp.fc2.weight',
 'encoder.layers.0.blocks.0.mlp.fc2.bias',
 'encoder.layers.0.blocks.1.norm1.weight',
 'encoder.layers.0.blocks.1.norm1.bias',
 'encoder.layers.0.blocks.1.attn.relative_position_bias_table',
 'encoder.layers.0.blocks.1.attn.relative_position_index',
 'en

In [19]:
path_ = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/swint_upernet_imp_sep_model.pth"
samrs_model.load_state_dict(torch.load(path_)['state_dict'])

<All keys matched successfully>

In [20]:
dummy = torch.randn(4,3,224,224)
samrs_model.forward(dummy)

ValueError: SyncBatchNorm expected input tensor to be on GPU

In [21]:
import dataset_git

In [24]:
dataset_ = dataset_git.ISAIDDataset(data_root = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/00.Data/isaid/TrainData", split = 'train')

Totally 0 samples in train set.


TypeError: Size should be int or sequence. Got <class 'NoneType'>